In [7]:

import re
import json
from wiktionaryparser import WiktionaryParser
parser = WiktionaryParser()

In [8]:
def fetch_wiki_word(word):
    Dict = {}
    try:
        response = parser.fetch(word)
        
        for meaning in response:
            # filtering pronunciations.
            IPAs = {}
            for p in meaning['pronunciations']['text']:
                if 'IPA:' in p:
                    UK = re.search('England|UK|Received Pronunciation', p)
                    US = re.search('US|American|GenAm|enPR', p)
                    if US and 'US' not in IPAs:
                        IPAs['US'] = re.search('(IPA: )(.*)',p).group(2)
                    if UK and 'UK' not in IPAs:
                        IPAs['UK'] = re.search('(IPA: )(.*)',p).group(2)
                    if re.match('IPA:',p) and 'IPA' not in IPAs: 
                        IPAs['IPA'] = re.search('(IPA: )(.*)',p).group(2)
            if len(IPAs) == 3:
                del IPAs['IPA']
            if len(IPAs) == 2:
                if 'UK' in IPAs and 'US' in IPAs:
                    if IPAs['UK'] == IPAs['US']:
                        IPAs['IPA'] = IPAs['US']  
                        del IPAs['UK'], IPAs['US']
                        
            for definition in meaning['definitions']:
                word_class = definition['partOfSpeech']
                def_list = []
                count = 0
                for df in definition['text']:
                    count += 1
                    if count == 1:
                        morfology = df
                        continue
                    match = re.search("(?:\(.*)(UK|slang|obsolete|dialectal|Caribbean|MLE)(?:.*\))", df)
                    if match:
                        continue
                    match = re.match("Dated form", df)
                    if match:
                        continue
                    def_list.append(df)  
                if len(def_list) == 0: 
                    continue
                if len(definition["examples"]) == 0: 
                    continue
                
                examples = []
                for Eg in definition['examples']:
                    if len(Eg) < 150 and not re.match('Audio',Eg) and not re.match("Synonym", Eg):
                        examples.append(Eg)
                if len(examples) == 0: 
                    continue
                examples.sort(key=lambda x: len(x))
                if len(examples) > 10:
                    examples = examples[:10]

                data = {}    
                data['class'] = word_class
                data['examples'] = examples
                    
                if word in Dict:       
                    Dict['meanings'].append(data) 
                else: 
                    Dict["morfology"] = morfology
                    Dict['pronunciation'] = IPAs
                    Dict['meanings'] = []
                    Dict['meanings'].append(data) 
    except:
        return {}

    return Dict

In [ ]:
example = fetch_wiki_word('fell')

with open("wiki_example.json", 'w') as f:
    json.dump(example, f, ensure_ascii = False)

In [4]:
import pandas as pd
common_words = pd.read_csv('common_words.csv')

In [5]:
common_words = common_words[common_words['word'].str.len() > 2]
list_of_common = common_words['word'].to_list()

In [6]:
with open("wordsList.json", 'r') as f:
    wordsList = json.loads(f.read())

In [7]:
new_list = list(set(list_of_common).difference(wordsList))

In [19]:
new_dict = {}

for word in word_list:
    print(word)
    new_dict[word] = fetch_wiki_word(word)


assistance
unfortunately
alright
intelligence
solve
baobab
tall
fault
forgive
he
opposite
slowly
important
everybody
sheep
pencil
colour
paint
departure
learnt
bushes


In [20]:

with open("wiki_data_trans.json", 'r') as f:
    old_dict = json.loads(f.read())

new_dict.update(old_dict)
with open("wiki_data_trans.json", 'w') as f:
    json.dump(new_dict, f, ensure_ascii = False)

In [16]:
txt = '''a
added
after
again
age
ago
all
already
alright
also
and
another
any
anything
as
asked
assistance
at
ate
back
baobab
before
best
big
bought
box
bushes
but
came
church
clear
clever
clothes
colour
come
could
course
day
departure
describe
detail
draw
drawing
during
each
eat
even
every
everybody
explained
fault
feel
forget
forgive
friend
from
grow
had
happened
hard
have
he
here
how
idea
if
important
in
information
intelligence
journey
know
large
laugh
learnt
life
like
little
long
look
made
make
maybe
memory
mistake
moment
never
nine
on
opposite
other
paint
past
pencil
picture
possible
problem
question
quite
result
right
sad
see
sheep
short
simple
single
six
size
slowly
small
solve
some
something
stand
start
sure
talking
tall
thank
that
then
third
thought
through
time
told
too
took
trees
tried
true
try
understand
unfortunately
use
usually
very
wall
want
with
without
would ''' 

In [17]:
txt = txt.splitlines()
txt = [x.strip() for x in txt]
txt

['a',
 'added',
 'after',
 'again',
 'age',
 'ago',
 'all',
 'already',
 'alright',
 'also',
 'and',
 'another',
 'any',
 'anything',
 'as',
 'asked',
 'assistance',
 'at',
 'ate',
 'back',
 'baobab',
 'before',
 'best',
 'big',
 'bought',
 'box',
 'bushes',
 'but',
 'came',
 'church',
 'clear',
 'clever',
 'clothes',
 'colour',
 'come',
 'could',
 'course',
 'day',
 'departure',
 'describe',
 'detail',
 'draw',
 'drawing',
 'during',
 'each',
 'eat',
 'even',
 'every',
 'everybody',
 'explained',
 'fault',
 'feel',
 'forget',
 'forgive',
 'friend',
 'from',
 'grow',
 'had',
 'happened',
 'hard',
 'have',
 'he',
 'here',
 'how',
 'idea',
 'if',
 'important',
 'in',
 'information',
 'intelligence',
 'journey',
 'know',
 'large',
 'laugh',
 'learnt',
 'life',
 'like',
 'little',
 'long',
 'look',
 'made',
 'make',
 'maybe',
 'memory',
 'mistake',
 'moment',
 'never',
 'nine',
 'on',
 'opposite',
 'other',
 'paint',
 'past',
 'pencil',
 'picture',
 'possible',
 'problem',
 'question',
 'q

In [18]:


with open("wiki_data_trans.json", 'r') as f:
    wiki_data = json.loads(f.read())

word_list = list(set(txt).difference(list(wiki_data.keys())))
word_list

['assistance',
 'unfortunately',
 'alright',
 'intelligence',
 'solve',
 'baobab',
 'tall',
 'fault',
 'forgive',
 'he',
 'opposite',
 'slowly',
 'important',
 'everybody',
 'sheep',
 'pencil',
 'colour',
 'paint',
 'departure',
 'learnt',
 'bushes']

In [4]:
a = ['a','b','d','c']
b = ['c','e','a']
c = list(set(a).difference(b))
c

['b', 'd']